# Processing mutations

The variant calls from the BreSeq pipeline are parsed and analysed. All strains with _mutX_ mutations are excluded as they accumulate SNPs at a very high rate, masking interesting mutations.

Mutations that are present in all strains are also excluded as they have most likely also been in the background strain, prior to the evolution experiments.

In [1]:
import pandas as pd
import json
import copy
import matplotlib.pyplot as plt
import numpy as np

In [2]:
%matplotlib inline

In [3]:
compounds = ["HMDA", "putrescine", "1,2-propanediol", "2,3-butanediol", "glutarate", "adipate", "hexanoate",
             "octanoate", "coumarate", "isobutyrate", "butanol", "no_chemical"
            ]

mapping = pd.read_csv("../Data/Mutation_data/Variant_calls/mapping.csv", header=None)
compound_mapping = {
    1: "butanol",
    2: "glutarate",
    3: "coumarate",
    5: "HMDA",
    6: "putrescine",
    7: "adipate",
    8: "isobutyrate",
    9: "hexanoate",
    10: "2,3-butanediol",
    11: "1,2-propanediol",
    12: "octanoate",
    13: "no_chemical"
}
rev_compound_mapping = {v: k for k, v in compound_mapping.items()}

In [4]:
code_to_compound = {
    "BUT": "butanol",
    "GLUT": "glutarate",
    "COUM": "coumarate",
    "HMDA": "HMDA",
    "PUTR": "putrescine",
    "ADIP": "adipate",
    "IBUA": "isobutyrate",
    "HEXA": "hexanoate",
    "23BD": "2,3-butanediol",
    "12PD": "1,2-propanediol",
    "OCTA": "octanoate",
    "GLU":"no_chemical"
}
comp_to_code = {v: k for k, v in code_to_compound.items()}

Generate a mutation id from the information from BreSeq.


In [5]:
def generate_mut_id(row):
    typ = row["Mutation Type"]
    pos = str(int(row["Position"].replace(",", "")))
    if typ == "SNP":
        last = row["Sequence Change"][-1]
    elif typ == "DEL":
        if "(" in row["Sequence Change"]:
            last = str(len(row["Sequence Change"].split(")")[0][1:]))
        else:
            last = str(int(row["Sequence Change"][1:-3].replace(",", "")))
    elif typ == "INS":
        if "(" in row["Sequence Change"]:
            last = row["Sequence Change"].split(")")[0][1:]
        else:
            last = row["Sequence Change"][1:]
    elif typ == "MOB":
        fields = row["Sequence Change"].split("\xa0")
        if len(fields)==1: #handling excdptions for Glu samples
            fields = str(fields[0]).split(" ")
        last = fields[0] + "-" + fields[2][1:]
    elif typ == "DUP":
        times = row["Sequence Change"].split("x")[-1]
        size = row["Sequence Change"].split(" bp ")[0]
        last = size + "_" + times
    else:
        last = ""
    return "-".join((typ, pos, last))

Convert the strain code from the breseq results to the real strain names using the mapping file.

In [6]:
def strain_code_to_name(strain_code, compound):
    compound_num = rev_compound_mapping[compound]
    fields = strain_code.split(" ")[-4:]
    fields = [f[1:] for f in fields]
    code = (int(fields[0]), int(fields[2]), int(fields[3]))
    strain = mapping[mapping[0] == compound_num].groupby(by=[1, 2, 3]).first().loc[code][4]
    return strain

In [7]:
def convert_to_int(string):
    if string[0] == "‑": # Weird non-standard minus sign
        string = "-" + string[1:] # Replace with normal minus sign
    return int(string)

def pick_gene(rel_coords):
    """Given relative coordinates of an intergenic mutation, pick the gene that is closest downstream"""
    if min(rel_coords) > 0:
        return None
    elif max(rel_coords) <= 0:
        if rel_coords[0] > rel_coords[1]:
            return 0
        else:
            return 1
    elif rel_coords[0] <= 0:
        return 0
    elif rel_coords[1] <= 0:
        return 1
    else:
        raise RuntimeError("What else is there?")

In [8]:
excluded_mutations = {
    'DEL-1299499-1199', # Randomly called across populations DEL ychE/oppA intergenic(+254/-485)
}

In [9]:
strain_to_muts = {}
mut_to_genes = {}
mut_to_change = {}
mut_to_seq = {}
mut_to_full_details = {}

for comp in compounds:
    df = pd.read_csv("../Data/Mutation_data/Variant_calls/%s.csv" % comp)    
    df["mut_id"] = df.apply(generate_mut_id, 1)
    
    for idx, row in df.iterrows():
        genes = row["Gene"]
        if "genes" in genes:
            genes = genes.split("genes")[-1] # Fix inconsistent format for large deletions
        genes = genes.split(", ")
        genes = [gene.strip("[] >") for gene in genes]
        change = row["Protein change"]
        if isinstance(change, str) and change.startswith("intergenic"):
            nums = change.split("(")[1][:-1].split("/")
            nums = convert_to_int(nums[0]), convert_to_int(nums[1])
            nearest = pick_gene(nums)
            if nearest is None:
                genes = []
            else:
                genes = [genes[nearest]]
                
        mut_id = row["mut_id"]
        mut_to_genes[mut_id] = genes
        mut_to_change[mut_id] = change
        mut_to_seq[mut_id] = row['Sequence Change']
        mut_to_full_details[mut_id] = row['Protein change']
        
    for strain_code in df.columns[10:]:
        if strain_code == "mut_id":
            continue
        strain = strain_code_to_name(strain_code, comp)
        if strain.endswith("-rerun"):
            strain = strain[:-6]

        muts = list(df[df[strain_code] == 1]["mut_id"])
        strain_to_muts[strain] = set(muts)  - excluded_mutations # "-rerun" samples overwrite original samples

In [10]:
with open("../Data/Mutation_data/Mutation_sequence_changes_glc.json", "w") as outfile:
    json.dump(mut_to_seq, outfile)
with open("../Data/Mutation_data/Mutation_protein_changes_glc.json", "w") as outfile:
    json.dump(mut_to_full_details, outfile)

In [11]:
# Save the protein effect of all SNP mutations
mut_effects = {}
for mut, change in mut_to_change.items():
    typ = mut.split("-")[0]
    if typ == "SNP":
        if "intergenic" in change:
            effect = "intergenic"
        elif "pseudogene" in change:
            effect = "pseudogene"
        else:
            subs = change.split()[0]
            effect = subs
    else:
        effect = ""
        if change == change and typ != "DUP":
            effect = change.split()[0]
            # print(typ, change)
    mut_effects[mut] = (typ, effect)
    
with open("../Data/Mutation_data/Mutation_effects_glc.json", "w") as outfile:
    json.dump(mut_effects, outfile)

In [12]:
strain_to_genes = {}

# Add genes that are specifically mutated (mutation only has 1 gene target)
for strain, muts in strain_to_muts.items():
    for mut in muts:
        if len(mut_to_genes[mut]) == 1:
            strain_to_genes.setdefault(strain, []).append(mut_to_genes[mut][0])
            
strain_to_genes = {k: set(v) for k, v in strain_to_genes.items()}

certain_mutations = {
    comp: set.union(*(v for k, v in strain_to_genes.items() if k.startswith(comp))) for comp in code_to_compound
}
strain_to_all_genes = copy.deepcopy(strain_to_genes)

# For mutations that affect multiple genes: only add genes that are already mutated in strains from that compound
# (or mutator genes)
for comp in code_to_compound:
    for strain in (s for s in strain_to_muts if s.startswith(comp)):
        for mut in strain_to_muts[strain]:
            if mut.startswith("DUP"):
                continue
            if len(mut_to_genes[mut]) > 1:
                for gene in mut_to_genes[mut]:
                    strain_to_all_genes[strain].add(gene)
                    if gene in certain_mutations[comp] or gene.startswith("mut"):
                        strain_to_genes[strain].add(gene)

In [13]:
# Hypermutators are strains with mutations in a mutX gene
hypermutators = [s for s, genes in strain_to_genes.items() if "mut" in set(g[:3] for g in genes)]
non_hypermutators = set(strain_to_genes) - set(hypermutators)

In [14]:
# Create an overview table of mutations in all strains
mut_df = pd.DataFrame({s: {m: 1 for m in muts} for s, muts in strain_to_muts.items()}).fillna(0).astype("int")
mut_df.to_csv("../Data/Mutation_data/strain_mutation_table_glc.tsv", sep="\t")

In [15]:
with open("../Data/Mutation_data/Strain_to_genes_glc.json", "w") as outfile:
    json.dump({k: list(v) for k, v in strain_to_genes.items() if k in non_hypermutators}, outfile)
    
with open("../Data/Mutation_data/Strain_to_all_genes_glc.json", "w") as outfile:
    json.dump({k: list(v) for k, v in strain_to_all_genes.items() if k in non_hypermutators}, outfile)
    
with open("../Data/Mutation_data/All_strains_to_gene_names_glc.json", "w") as outfile:
    json.dump({k: list(v) for k, v in strain_to_genes.items()}, outfile)
    
with open("../Data/Mutation_data/All_strains_to_mutations_glc.json", "w") as outfile:
    json.dump({k: list(v) for k, v in strain_to_muts.items()}, outfile)
    
with open("../Data/Mutation_data/All_mutated_genes_glc.txt", "w") as outfile:
    mutated_genes = set.union(*[strain_to_genes[s] for s in non_hypermutators])
    outfile.write("\n".join(mutated_genes))
    
with open("../Data/Mutation_data/Mutations_to_gene_names_glc.json", "w") as outfile:
    json.dump({k: v for k, v in mut_to_genes.items()}, outfile)

In [16]:
hypermutators_df = pd.DataFrame(hypermutators, columns=["strain_code"])
hypermutators_df.to_csv("../Data/Mutation_data/hypermutators_list_glc.csv",index=False)

In [17]:
mapping.to_csv("../Data/Mutation_data/Variant_calls/mapping_glc.csv")